<a href="https://colab.research.google.com/github/John1495/RNA-3-D-1/blob/main/GVP_model_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.1.0+cpu.html


Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.4/500.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.0 MB/s eta 0:00:00


In [22]:
!pip install wandb
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [3]:
!pip install numpy matplotlib py3Dmol


In [7]:
!pip uninstall torch
!pip install torch==<your_version>+cpu -f https://download.pytorch.org/whl/torch_stable.html

Found existing installation: torch 2.6.0+cpu
Uninstalling torch-2.6.0+cpu:
  Would remove:
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.11/dist-packages/functorch/*
    /usr/local/lib/python3.11/dist-packages/torch-2.6.0+cpu.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torch/*
    /usr/local/lib/python3.11/dist-packages/torchgen/*
Proceed (Y/n)? ERROR: Operation cancelled by user
/bin/bash: line 1: your_version: No such file or directory


In [4]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.9/583.9 kB 38.9 MB/s eta 0:00:00


In [5]:
!pip install torch torch_geometric optuna matplotlib biopython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 36.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd

sequences_df = pd.read_csv('/kaggle/cleaned_train_sequences2 (1).csv')
labels_df = pd.read_csv('/kaggle/train_labels1.csv')

print("Sequences:")
print(sequences_df.head(3))

print("\nLabels:")
print(labels_df.head(3))


Sequences:
  target_id                            sequence temporal_cutoff  \
0    1SCL_A       GGGUGCUCAGUACGAGAGGAACCGCACCC      1995-01-26   
1    1RNK_A  GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU      1995-02-27   
2    1RHT_A            GGGACUGACGAUCACGCAGUCUAU      1995-06-03   

                                         description  \
0               THE SARCIN-RICIN LOOP, A MODULAR RNA   
1  THE STRUCTURE OF AN RNA PSEUDOKNOT THAT CAUSES...   
2  24-MER RNA HAIRPIN COAT PROTEIN BINDING SITE F...   

                                       all_sequences  
0  >1SCL_1|Chain A|RNA SARCIN-RICIN LOOP|Rattus n...  
1  >1RNK_1|Chain A|RNA PSEUDOKNOT|null\nGGCGCAGUG...  
2  >1RHT_1|Chain A|RNA (5'-R(P*GP*GP*GP*AP*CP*UP*...  

Labels:
   resid         ID resname        x_1        y_1    z_1
0      1   17RA_A_1       G  35.856998 -10.769000 -7.548
1     10  17RA_A_10       A  26.341999  12.365000 -0.594
2     11  17RA_A_11       U  23.917999  16.023001 -5.418


In [ ]:
import os
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv
import numpy as np


# Dataset class to load RNA sequence and labels
class RNAGraphDataset(Dataset):
    def __init__(self, sequences_file, labels_file):
        self.sequences = pd.read_csv(sequences_file)
        self.labels = pd.read_csv(labels_file)

        self.graphs = self.build_graphs()

    def build_graphs(self):
        graphs = []
        label_dict = self.labels.groupby("ID")

        for _, row in self.sequences.iterrows():
            target_id = row['target_id']
            sequence = row['sequence']
            node_feats = self.encode_sequence(sequence)

            # Match label
            matched_labels = self.labels[self.labels['ID'].str.startswith(target_id)]
            if matched_labels.empty or len(matched_labels) != len(sequence):
                continue  # Skip mismatched

            coords = matched_labels[['x_1', 'y_1', 'z_1']].values.astype(np.float32)

            # Edge index (chain sequentially)
            edge_index = self.build_edge_index(len(sequence))

            data = Data(
                x=torch.tensor(node_feats, dtype=torch.float),
                edge_index=edge_index,
                y=torch.tensor(coords, dtype=torch.float)
            )
            graphs.append(data)

        return graphs

    def encode_sequence(self, sequence):
        mapping = {'A': [1,0,0,0], 'U': [0,1,0,0], 'G': [0,0,1,0], 'C': [0,0,0,1]}
        return [mapping.get(nt, [0,0,0,0]) for nt in sequence]

    def build_edge_index(self, length):
        edge_index = []
        for i in range(length - 1):
            edge_index.append([i, i+1])
            edge_index.append([i+1, i])  # Add bidirectional edges
        return torch.tensor(edge_index).t().contiguous()

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]


# GVP-like model using GCNConv as a placeholder
class GVPModel(torch.nn.Module):
    def __init__(self, in_channels=4, hidden_channels=64):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, 3)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.lin(x)
        return x


# Custom collate function to handle batching of Data objects
def collate_fn(batch):
    return Batch.from_data_list(batch)


# Training function
def train(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        pred = model(data)
        loss = F.mse_loss(pred, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


# Evaluation function
def evaluate(model, loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            pred = model(data)
            loss = F.mse_loss(pred, data.y)
            total_loss += loss.item()
    return total_loss / len(loader)


# Main function to run training and validation
def main():
    # File paths for train and validation data
    train_sequences = '/kaggle/cleaned_train_sequences2 (1).csv'
    train_labels = '/kaggle/train_labels1.csv'
    validation_sequences = '/kaggle/validation_sequences.csv'
    validation_labels = '/kaggle/validation_labels.csv'

    # Load the dataset
    dataset = RNAGraphDataset(train_sequences, train_labels)
    if len(dataset) == 0:
        raise ValueError("No valid training data found. Check sequence-label matching.")

    # Random split the dataset into train and validation sets
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    # Create DataLoader for train and validation sets
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=1, collate_fn=collate_fn)

    # Setup device and model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GVPModel().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # Track the best model based on validation loss
    best_val_loss = float('inf')
    for epoch in range(1, 101):
        train_loss = train(model, train_loader, optimizer, device)
        val_loss = evaluate(model, val_loader, device)
        print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

        # Save the model with the best validation loss
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_gvp_model.pt")


if __name__ == "__main__":
    main()


Epoch 1: Train Loss = 9696.4653, Val Loss = 9952.4718
Epoch 2: Train Loss = 8930.3638, Val Loss = 9792.6099
Epoch 3: Train Loss = 8913.2559, Val Loss = 9873.7861
Epoch 4: Train Loss = 8909.9442, Val Loss = 9770.0099
Epoch 5: Train Loss = 8911.4973, Val Loss = 9890.3780
Epoch 6: Train Loss = 8864.6202, Val Loss = 9721.2444
Epoch 7: Train Loss = 8925.6557, Val Loss = 9812.3279
Epoch 8: Train Loss = 8903.7461, Val Loss = 9838.0926
Epoch 9: Train Loss = 8900.9657, Val Loss = 9889.6365
Epoch 10: Train Loss = 8896.2538, Val Loss = 9728.6678
Epoch 11: Train Loss = 8899.6930, Val Loss = 9822.0760
Epoch 12: Train Loss = 8904.7194, Val Loss = 9904.1818
Epoch 13: Train Loss = 8901.4221, Val Loss = 9733.9354
Epoch 14: Train Loss = 8892.1657, Val Loss = 9923.2250
Epoch 15: Train Loss = 8902.3542, Val Loss = 9939.7261
Epoch 16: Train Loss = 8912.3565, Val Loss = 9854.1458
Epoch 17: Train Loss = 8908.6952, Val Loss = 9818.4915
Epoch 18: Train Loss = 8903.2166, Val Loss = 9831.9160
Epoch 19: Train Los

In [2]:
import os
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv
import numpy as np


# Dataset class to load RNA sequence and labels
class RNAGraphDataset(Dataset):
    def __init__(self, sequences_file, labels_file):
        self.sequences = pd.read_csv(sequences_file)
        self.labels = pd.read_csv(labels_file)

        self.mean = None
        self.std = None

        self.graphs = self.build_graphs()

    def build_graphs(self):
        graphs = []
        label_dict = self.labels.groupby("ID")

        all_coords = []
        # First pass to collect stats
        for _, row in self.sequences.iterrows():
            target_id = row['target_id']
            sequence = row['sequence']
            matched_labels = self.labels[self.labels['ID'].str.startswith(target_id)]
            if matched_labels.empty or len(matched_labels) != len(sequence):
                continue
            coords = matched_labels[['x_1', 'y_1', 'z_1']].values.astype(np.float32)
            all_coords.append(coords)

        all_coords = np.concatenate(all_coords, axis=0)
        self.mean = all_coords.mean(axis=0)
        self.std = all_coords.std(axis=0)

        # Second pass to build graphs
        for _, row in self.sequences.iterrows():
            target_id = row['target_id']
            sequence = row['sequence']
            node_feats = self.encode_sequence(sequence)
            matched_labels = self.labels[self.labels['ID'].str.startswith(target_id)]
            if matched_labels.empty or len(matched_labels) != len(sequence):
                continue
            coords = matched_labels[['x_1', 'y_1', 'z_1']].values.astype(np.float32)
            coords = (coords - self.mean) / self.std
            edge_index = self.build_edge_index(len(sequence))
            data = Data(
                x=torch.tensor(node_feats, dtype=torch.float),
                edge_index=edge_index,
                y=torch.tensor(coords, dtype=torch.float)
            )
            graphs.append(data)
        return graphs

    def encode_sequence(self, sequence):
        mapping = {'A': [1,0,0,0], 'U': [0,1,0,0], 'G': [0,0,1,0], 'C': [0,0,0,1]}
        return [mapping.get(nt, [0,0,0,0]) for nt in sequence]

    def build_edge_index(self, length):
        edge_index = []
        for i in range(length - 1):
            edge_index.append([i, i+1])
            edge_index.append([i+1, i])
        return torch.tensor(edge_index).t().contiguous()

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]


# Enhanced GVP-like model using GCNConv
class GVPModel(torch.nn.Module):
    def __init__(self, in_channels=4, hidden_channels=64):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.dropout = torch.nn.Dropout(0.3)
        self.lin1 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, 3)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.dropout(F.relu(self.conv1(x, edge_index)))
        x = self.dropout(F.relu(self.conv2(x, edge_index)))
        x = self.dropout(F.relu(self.conv3(x, edge_index)))
        x = self.dropout(F.relu(self.lin1(x)))
        x = self.lin2(x)
        return x


# Custom collate function to handle batching

def collate_fn(batch):
    return Batch.from_data_list(batch)


# Training function with gradient clipping

def train(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        pred = model(data)
        loss = F.mse_loss(pred, data.y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


# Evaluation function

def evaluate(model, loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            pred = model(data)
            loss = F.mse_loss(pred, data.y)
            total_loss += loss.item()
    return total_loss / len(loader)


# Main training loop

def main():
    train_sequences = '/kaggle/cleaned_train_sequences2 (1).csv'
    train_labels = '/kaggle/train_labels1.csv'
    validation_sequences = '/kaggle/validation_sequences.csv'
    validation_labels = '/kaggle/validation_labels.csv'

    dataset = RNAGraphDataset(train_sequences, train_labels)
    if len(dataset) == 0:
        raise ValueError("No valid training data found. Check sequence-label matching.")

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=4, collate_fn=collate_fn)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GVPModel().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    best_val_loss = float('inf')
    patience_counter = 0
    for epoch in range(1, 101):
        train_loss = train(model, train_loader, optimizer, device)
        val_loss = evaluate(model, val_loader, device)
        scheduler.step(val_loss)

        print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), "best_gvp_model.pt")
        else:
            patience_counter += 1
            if patience_counter > 10:
                print("Early stopping due to no improvement in validation loss.")
                break


if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_spline_conv/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOp

Epoch 1: Train Loss = 0.7275, Val Loss = 0.7240
Epoch 2: Train Loss = 0.6815, Val Loss = 0.7248
Epoch 3: Train Loss = 0.7296, Val Loss = 0.7246
Epoch 4: Train Loss = 0.7082, Val Loss = 0.7335
Epoch 5: Train Loss = 0.7271, Val Loss = 0.7211
Epoch 6: Train Loss = 0.7620, Val Loss = 0.7262
Epoch 7: Train Loss = 0.7057, Val Loss = 0.7250
Epoch 8: Train Loss = 0.7533, Val Loss = 0.7227
Epoch 9: Train Loss = 0.7422, Val Loss = 0.7225
Epoch 10: Train Loss = 0.6937, Val Loss = 0.7221
Epoch 11: Train Loss = 0.7104, Val Loss = 0.7201
Epoch 12: Train Loss = 0.7302, Val Loss = 0.7201
Epoch 13: Train Loss = 0.7182, Val Loss = 0.7238
Epoch 14: Train Loss = 0.7485, Val Loss = 0.7239
Epoch 15: Train Loss = 0.7336, Val Loss = 0.7217
Epoch 16: Train Loss = 0.7581, Val Loss = 0.7210
Epoch 17: Train Loss = 0.7525, Val Loss = 0.7294
Epoch 18: Train Loss = 0.6777, Val Loss = 0.7232
Epoch 19: Train Loss = 0.7319, Val Loss = 0.7220
Epoch 20: Train Loss = 0.6822, Val Loss = 0.7191
Epoch 21: Train Loss = 0.7505

In [16]:
#!/usr/bin/env python3
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from torch.optim.lr_scheduler import OneCycleLR
import warnings

# Suppress unnecessary warnings
warnings.filterwarnings("ignore", category=UserWarning)

# ----------------------------
# Dataset Preparation
# ----------------------------
class RNAGraphDataset(Dataset):
    def __init__(self, sequences_file, labels_file):
        self.sequences = pd.read_csv(sequences_file)
        self.labels = pd.read_csv(labels_file)
        self.mean = None
        self.std = None
        self.graphs = self._process_data()

    def _process_data(self):
        graphs = []
        all_coords = []

        # First pass: collect statistics
        for _, row in self.sequences.iterrows():
            target_id = row['target_id']
            matched_labels = self.labels[self.labels['ID'].str.startswith(target_id)]
            if not matched_labels.empty and len(matched_labels) == len(row['sequence']):
                coords = matched_labels[['x_1', 'y_1', 'z_1']].values.astype(np.float32)
                all_coords.append(coords)

        all_coords = np.concatenate(all_coords, axis=0)
        self.mean = all_coords.mean(axis=0)
        self.std = all_coords.std(axis=0) + 1e-8  # Prevent division by zero

        # Second pass: build graphs
        for _, row in self.sequences.iterrows():
            target_id = row['target_id']
            sequence = row['sequence']
            matched_labels = self.labels[self.labels['ID'].str.startswith(target_id)]

            if matched_labels.empty or len(matched_labels) != len(sequence):
                continue

            node_feats = self._encode_sequence(sequence)
            coords = matched_labels[['x_1', 'y_1', 'z_1']].values.astype(np.float32)
            coords = (coords - self.mean) / self.std

            edge_index = self._build_edges(len(sequence))

            graphs.append(Data(
                x=torch.tensor(node_feats, dtype=torch.float32),
                edge_index=edge_index,
                y=torch.tensor(coords, dtype=torch.float32)
            ))
        return graphs

    def _encode_sequence(self, sequence):
        mapping = {'A': [1,0,0,0], 'U': [0,1,0,0],
                  'G': [0,0,1,0], 'C': [0,0,0,1]}
        return [mapping.get(nt, [0,0,0,0]) for nt in sequence]

    def _build_edges(self, length):
        edge_index = []
        for i in range(length - 1):
            # Add bidirectional edges
            edge_index.append([i, i+1])
            edge_index.append([i+1, i])
        return torch.tensor(edge_index).t().contiguous()

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]

# ----------------------------
# Enhanced GNN Model (Fixed)
# ----------------------------
class RNA3DModel(nn.Module):
    def __init__(self, in_channels=4, hidden_channels=256, num_layers=4):
        super().__init__()
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()

        # Input layer
        self.convs.append(GCNConv(in_channels, hidden_channels))
        self.bns.append(nn.BatchNorm1d(hidden_channels))

        # Hidden layers
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            self.bns.append(nn.BatchNorm1d(hidden_channels))

        # Output layer
        self.convs.append(GCNConv(hidden_channels, hidden_channels))
        self.fc = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels//2),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels//2, 3)
        )

        self.dropout = nn.Dropout(0.4)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Message passing without edge attributes
        for i, (conv, bn) in enumerate(zip(self.convs, self.bns)):
            x = conv(x, edge_index)
            if i != len(self.convs) - 1:
                x = bn(x)
                x = F.leaky_relu(x, 0.2)
                x = self.dropout(x)

        return self.fc(x)

# ----------------------------
# Training Utilities
# ----------------------------
def collate_fn(batch):
    return Batch.from_data_list(batch)

def train_epoch(model, loader, optimizer, device, grad_accum=4):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for i, data in enumerate(loader):
        data = data.to(device)
        pred = model(data)
        loss = F.mse_loss(pred, data.y) / grad_accum
        loss.backward()

        if (i + 1) % grad_accum == 0:
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()

        total_loss += loss.item() * grad_accum

    return total_loss / len(loader)

def validate(model, loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            pred = model(data)
            total_loss += F.mse_loss(pred, data.y).item()
    return total_loss / len(loader)

# ----------------------------
# Main Training Loop
# ----------------------------
def main():
    # File paths (updated with your Kaggle paths)
    train_sequences = '/kaggle/cleaned_train_sequences2 (1).csv'
    train_labels = '/kaggle/train_labels1.csv'
    validation_sequences = '/kaggle/validation_sequences.csv'
    validation_labels = '/kaggle/validation_labels.csv'

    # Configuration
    config = {
        'batch_size': 16,
        'hidden_dim': 256,
        'num_layers': 4,
        'max_epochs': 100,
        'patience': 20,
        'min_delta': 0.001,
        'grad_accum': 4
    }

    # Device setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Data loading - using separate train/validation files
    train_dataset = RNAGraphDataset(train_sequences, train_labels)
    val_dataset = RNAGraphDataset(validation_sequences, validation_labels)

    if len(train_dataset) == 0 or len(val_dataset) == 0:
        raise ValueError("No valid training/validation data found!")

    train_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        collate_fn=collate_fn
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        collate_fn=collate_fn
    )

    # Model setup
    model = RNA3DModel(
        hidden_channels=config['hidden_dim'],
        num_layers=config['num_layers']
    ).to(device)
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Optimizer and scheduler
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=1e-3,
        weight_decay=1e-4
    )
    scheduler = OneCycleLR(
        optimizer,
        max_lr=5e-3,
        steps_per_epoch=len(train_loader),
        epochs=config['max_epochs'],
        pct_start=0.2
    )

    # Training loop
    best_val = float('inf')
    patience = config['patience']

    for epoch in range(1, config['max_epochs'] + 1):
        train_loss = train_epoch(
            model, train_loader, optimizer, device,
            grad_accum=config['grad_accum']
        )
        val_loss = validate(model, val_loader, device)
        scheduler.step()

        print(f"Epoch {epoch:03d} | "
              f"Train Loss: {train_loss:.6f} | "
              f"Val Loss: {val_loss:.6f} | "
              f"LR: {optimizer.param_groups[0]['lr']:.2e}")

        # Early stopping check
        if val_loss < (best_val - config['min_delta']):
            best_val = val_loss
            patience = config['patience']
            torch.save(model.state_dict(), "best_model.pt")
            print(f"New best model saved (Val Loss: {val_loss:.6f})")
        else:
            patience -= 1
            if patience <= 0:
                print(f"Early stopping at epoch {epoch}")
                break

if __name__ == "__main__":
    main()

Using device: cpu
Model parameters: 233,475
Epoch 001 | Train Loss: 163.425323 | Val Loss: 10.179984 | LR: 2.00e-04
New best model saved (Val Loss: 10.179984)
Epoch 002 | Train Loss: 112.385022 | Val Loss: 18.906967 | LR: 2.00e-04
Epoch 003 | Train Loss: 88.193440 | Val Loss: 10.859036 | LR: 2.00e-04
Epoch 004 | Train Loss: 69.011355 | Val Loss: 9.162365 | LR: 2.00e-04
New best model saved (Val Loss: 9.162365)
Epoch 005 | Train Loss: 54.302818 | Val Loss: 7.411708 | LR: 2.00e-04
New best model saved (Val Loss: 7.411708)
Epoch 006 | Train Loss: 42.759427 | Val Loss: 5.990111 | LR: 2.00e-04
New best model saved (Val Loss: 5.990111)
Epoch 007 | Train Loss: 33.843796 | Val Loss: 4.987537 | LR: 2.01e-04
New best model saved (Val Loss: 4.987537)
Epoch 008 | Train Loss: 26.763395 | Val Loss: 4.120634 | LR: 2.01e-04
New best model saved (Val Loss: 4.120634)
Epoch 009 | Train Loss: 21.307909 | Val Loss: 4.186143 | LR: 2.01e-04
Epoch 010 | Train Loss: 17.164058 | Val Loss: 3.697249 | LR: 2.01e-0

In [18]:
#!/usr/bin/env python3
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from torch.optim.lr_scheduler import OneCycleLR
import warnings

# Suppress unnecessary warnings
warnings.filterwarnings("ignore", category=UserWarning)

# ----------------------------
# Enhanced Dataset with Edge Features
# ----------------------------
class RNAGraphDataset(Dataset):
    def __init__(self, sequences_file, labels_file):
        self.sequences = pd.read_csv(sequences_file)
        self.labels = pd.read_csv(labels_file)
        self.mean = None
        self.std = None
        self.graphs = self._process_data()

    def _process_data(self):
        graphs = []
        all_coords = []

        # First pass: collect statistics
        for _, row in self.sequences.iterrows():
            target_id = row['target_id']
            matched_labels = self.labels[self.labels['ID'].str.startswith(target_id)]
            if not matched_labels.empty and len(matched_labels) == len(row['sequence']):
                coords = matched_labels[['x_1', 'y_1', 'z_1']].values.astype(np.float32)
                all_coords.append(coords)

        all_coords = np.concatenate(all_coords, axis=0)
        self.mean = all_coords.mean(axis=0)
        self.std = all_coords.std(axis=0) + 1e-8

        # Second pass: build graphs with enhanced features
        for _, row in self.sequences.iterrows():
            target_id = row['target_id']
            sequence = row['sequence']
            matched_labels = self.labels[self.labels['ID'].str.startswith(target_id)]

            if matched_labels.empty or len(matched_labels) != len(sequence):
                continue

            node_feats = self._encode_sequence(sequence)
            coords = matched_labels[['x_1', 'y_1', 'z_1']].values.astype(np.float32)
            coords = (coords - self.mean) / self.std

            edge_index, edge_attr = self._build_edges(len(sequence), coords)

            graphs.append(Data(
                x=torch.tensor(node_feats, dtype=torch.float32),
                edge_index=edge_index,
                edge_attr=edge_attr,
                y=torch.tensor(coords, dtype=torch.float32),
                pos=torch.tensor(coords, dtype=torch.float32)  # Added positional encoding
            ))
        return graphs

    def _encode_sequence(self, sequence):
        # Enhanced nucleotide encoding
        mapping = {
            'A': [1,0,0,0, 0.1],  # Added chemical property dimension
            'U': [0,1,0,0, 0.2],
            'G': [0,0,1,0, 0.3],
            'C': [0,0,0,1, 0.4]
        }
        return [mapping.get(nt, [0,0,0,0, 0.0]) for nt in sequence]

    def _build_edges(self, length, coords):
        edge_index = []
        edge_attr = []
        for i in range(length - 1):
            # Bidirectional edges with distance features
            edge_index.append([i, i+1])
            edge_index.append([i+1, i])
            dist = np.linalg.norm(coords[i+1] - coords[i])
            edge_attr.extend([
                [dist, 1.0],  # Forward edge
                [dist, 0.0]   # Backward edge
            ])
        return (torch.tensor(edge_index).t().contiguous(),
                torch.tensor(np.array(edge_attr), dtype=torch.float32))

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]

# ----------------------------
# Enhanced GNN Model Architecture
# ----------------------------
class RNA3DModel(nn.Module):
    def __init__(self, in_channels=5, hidden_channels=512, num_layers=6):
        super().__init__()

        # Enhanced input projection
        self.input_proj = nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.1)
        )

        # Deeper GCN layers with residual connections
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        self.res_proj = nn.ModuleList()

        for i in range(num_layers):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            self.bns.append(nn.BatchNorm1d(hidden_channels))
            self.res_proj.append(nn.Linear(hidden_channels, hidden_channels) if i > 0 else None)

        # Enhanced output module
        self.output_net = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels*2),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),
            nn.Linear(hidden_channels*2, hidden_channels),
            nn.LeakyReLU(0.1),
            nn.Linear(hidden_channels, 3)
        )

        self.dropout = nn.Dropout(0.3)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        # Initial projection
        x = self.input_proj(x)

        # Message passing with residual connections
        for i, (conv, bn) in enumerate(zip(self.convs, self.bns)):
            res = x if self.res_proj[i] is None else self.res_proj[i](x)
            x = conv(x, edge_index) + res
            x = bn(x)
            x = F.leaky_relu(x, 0.1)
            x = self.dropout(x)

        return self.output_net(x)

# ----------------------------
# Optimized Training Utilities
# ----------------------------
def collate_fn(batch):
    return Batch.from_data_list(batch)

def train_epoch(model, loader, optimizer, device, grad_accum=4):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for i, data in enumerate(loader):
        data = data.to(device)
        pred = model(data)
        loss = F.mse_loss(pred, data.y) / grad_accum
        loss.backward()

        if (i + 1) % grad_accum == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)  # Tighter gradient clipping
            optimizer.step()
            optimizer.zero_grad()

        total_loss += loss.item() * grad_accum

    return total_loss / len(loader)

def validate(model, loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            pred = model(data)
            total_loss += F.mse_loss(pred, data.y).item()
    return total_loss / len(loader)

# ----------------------------
# Enhanced Main Training Loop
# ----------------------------
def main():
    # Configuration
    config = {
        'train_sequences': '/kaggle/cleaned_train_sequences2 (1).csv',
        'train_labels': '/kaggle/train_labels1.csv',
        'validation_sequences': '/kaggle/validation_sequences.csv',
        'validation_labels': '/kaggle/validation_labels.csv',
        'batch_size': 32 if torch.cuda.is_available() else 8,
        'hidden_dim': 512,
        'num_layers': 6,
        'max_epochs': 150,
        'patience': 25,
        'min_delta': 0.0005,
        'grad_accum': 4
    }

    # Device setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    if str(device) == 'cpu':
        print("WARNING: Training on CPU will be slow! Consider using GPU acceleration")

    # Data loading
    train_dataset = RNAGraphDataset(config['train_sequences'], config['train_labels'])
    val_dataset = RNAGraphDataset(config['validation_sequences'], config['validation_labels'])

    if len(train_dataset) == 0 or len(val_dataset) == 0:
        raise ValueError("No valid training/validation data found!")

    train_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=4 if str(device) == 'cuda' else 0
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        collate_fn=collate_fn,
        num_workers=4 if str(device) == 'cuda' else 0
    )

    # Model setup
    model = RNA3DModel(
        hidden_channels=config['hidden_dim'],
        num_layers=config['num_layers']
    ).to(device)
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Optimizer and scheduler
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=5e-4,
        weight_decay=1e-5
    )
    scheduler = OneCycleLR(
        optimizer,
        max_lr=1e-3,
        steps_per_epoch=len(train_loader),
        epochs=config['max_epochs'],
        pct_start=0.3,
        anneal_strategy='cos'
    )

    # Training loop
    best_val = float('inf')
    patience = config['patience']

    for epoch in range(1, config['max_epochs'] + 1):
        train_loss = train_epoch(model, train_loader, optimizer, device, config['grad_accum'])
        val_loss = validate(model, val_loader, device)
        scheduler.step()

        print(f"Epoch {epoch:03d} | "
              f"Train Loss: {train_loss:.6f} | "
              f"Val Loss: {val_loss:.6f} | "
              f"LR: {optimizer.param_groups[0]['lr']:.2e}")

        # Enhanced early stopping
        if val_loss < (best_val - config['min_delta']):
            best_val = val_loss
            patience = config['patience']
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_loss,
            }, "best_model.pt")
            print(f"New best model saved (Val Loss: {val_loss:.6f})")
        else:
            patience -= 1
            if patience <= 0:
                print(f"\nEarly stopping triggered at epoch {epoch}")
                print(f"Best validation loss achieved: {best_val:.6f}")
                break

    print("\nTraining completed!")

if __name__ == "__main__":
    main()

Using device: cpu
Model parameters: 3,950,083
Epoch 001 | Train Loss: 157.879368 | Val Loss: 7.729547 | LR: 4.00e-05
New best model saved (Val Loss: 7.729547)
Epoch 002 | Train Loss: 89.482609 | Val Loss: 12.405952 | LR: 4.00e-05
Epoch 003 | Train Loss: 58.264524 | Val Loss: 9.015454 | LR: 4.00e-05
Epoch 004 | Train Loss: 40.596371 | Val Loss: 5.506173 | LR: 4.00e-05
New best model saved (Val Loss: 5.506173)
Epoch 005 | Train Loss: 29.989074 | Val Loss: 4.281924 | LR: 4.00e-05
New best model saved (Val Loss: 4.281924)
Epoch 006 | Train Loss: 22.758186 | Val Loss: 2.943737 | LR: 4.00e-05
New best model saved (Val Loss: 2.943737)
Epoch 007 | Train Loss: 17.732665 | Val Loss: 2.280376 | LR: 4.00e-05
New best model saved (Val Loss: 2.280376)
Epoch 008 | Train Loss: 14.436116 | Val Loss: 1.674589 | LR: 4.00e-05
New best model saved (Val Loss: 1.674589)
Epoch 009 | Train Loss: 11.752934 | Val Loss: 1.310864 | LR: 4.00e-05
New best model saved (Val Loss: 1.310864)
Epoch 010 | Train Loss: 9.61

In [24]:
#!/usr/bin/env python3
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv
import numpy as np
from torch.optim.lr_scheduler import OneCycleLR
import warnings
from torch.nn import MultiheadAttention

# Suppress unnecessary warnings
warnings.filterwarnings("ignore", category=UserWarning)

# ----------------------------
# Enhanced Dataset with Geometric Features
# ----------------------------
class RNAGraphDataset(Dataset):
    def __init__(self, sequences_file, labels_file):
        self.sequences = pd.read_csv(sequences_file)
        self.labels = pd.read_csv(labels_file)
        self.mean = None
        self.std = None
        self.graphs = self._process_data()

    def _process_data(self):
        graphs = []
        all_coords = []

        # First pass: collect statistics
        for _, row in self.sequences.iterrows():
            target_id = row['target_id']
            matched_labels = self.labels[self.labels['ID'].str.startswith(target_id)]
            if not matched_labels.empty and len(matched_labels) == len(row['sequence']):
                coords = matched_labels[['x_1', 'y_1', 'z_1']].values.astype(np.float32)
                all_coords.append(coords)

        all_coords = np.concatenate(all_coords, axis=0)
        self.mean = all_coords.mean(axis=0)
        self.std = all_coords.std(axis=0) + 1e-8

        # Second pass: build graphs
        for _, row in self.sequences.iterrows():
            target_id = row['target_id']
            sequence = row['sequence']
            matched_labels = self.labels[self.labels['ID'].str.startswith(target_id)]

            if matched_labels.empty or len(matched_labels) != len(sequence):
                continue

            node_feats = self._encode_sequence(sequence)
            coords = matched_labels[['x_1', 'y_1', 'z_1']].values.astype(np.float32)
            coords = (coords - self.mean) / self.std

            edge_index, edge_attr = self._build_edges(len(sequence), coords)

            graphs.append(Data(
                x=torch.tensor(node_feats, dtype=torch.float32),
                edge_index=edge_index,
                edge_attr=edge_attr,
                y=torch.tensor(coords, dtype=torch.float32),
                pos=torch.tensor(coords, dtype=torch.float32)
            ))
        return graphs

    def _encode_sequence(self, sequence):
        mapping = {
            'A': [1,0,0,0, 0.12, 0.89],
            'U': [0,1,0,0, 0.23, 0.76],
            'G': [0,0,1,0, 0.34, 0.65],
            'C': [0,0,0,1, 0.45, 0.54]
        }
        return [mapping.get(nt, [0]*6) for nt in sequence]

    def _build_edges(self, length, coords):
        edge_index = []
        edge_attr = []
        for i in range(length - 1):
            vec = coords[i+1] - coords[i]
            dist = np.linalg.norm(vec)
            edge_index.append([i, i+1])
            edge_index.append([i+1, i])
            edge_attr.extend([
                [dist, vec[0], vec[1], vec[2], 1.0],
                [dist, -vec[0], -vec[1], -vec[2], 0.0]
            ])
        return (torch.tensor(edge_index).t().contiguous(),
                torch.tensor(np.array(edge_attr), dtype=torch.float32))

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]

# ----------------------------
# Enhanced GNN Model
# ----------------------------
class RNA3DModel(nn.Module):
    def __init__(self, in_channels=6, hidden_channels=512, num_layers=6):
        super().__init__()

        self.input_proj = nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.1)
        )

        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()
        for _ in range(num_layers):
            self.convs.append(GCNConv(hidden_channels, hidden_channels, improved=True))
            self.bns.append(nn.BatchNorm1d(hidden_channels))

        self.attn = MultiheadAttention(hidden_channels, num_heads=4, dropout=0.1)
        self.attn_ln = nn.LayerNorm(hidden_channels)

        self.output_net = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels*2),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),
            nn.Linear(hidden_channels*2, hidden_channels),
            nn.LeakyReLU(0.1),
            nn.Linear(hidden_channels, 3)
        )

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.input_proj(x)

        for conv, bn in zip(self.convs, self.bns):
            x = conv(x, edge_index) + x
            x = bn(x)
            x = F.leaky_relu(x, 0.1)

        attn_out, _ = self.attn(x.unsqueeze(0), x.unsqueeze(0), x.unsqueeze(0))
        x = x + self.attn_ln(attn_out.squeeze(0))

        return self.output_net(x)

# ----------------------------
# Training Utilities
# ----------------------------
def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0

    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()

        pred = model(data)
        loss = F.mse_loss(pred, data.y)

        # Regularization
        l2_reg = 0.001 * sum(p.norm(2) for p in model.parameters())
        l1_reg = 0.0001 * sum(p.abs().sum() for p in model.parameters())
        loss += l2_reg + l1_reg

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

def validate(model, loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            pred = model(data)
            total_loss += F.mse_loss(pred, data.y).item()
    return total_loss / len(loader)

# ----------------------------
# Main Training Loop
# ----------------------------
def main():
    # Configuration
    config = {
        "train_sequences": "/kaggle/cleaned_train_sequences2 (1).csv",
        "train_labels": "/kaggle/train_labels1.csv",
        "validation_sequences": "/kaggle/validation_sequences.csv",
        "validation_labels": "/kaggle/validation_labels.csv",
        "batch_size": 32 if torch.cuda.is_available() else 8,
        "hidden_dim": 512,
        "num_layers": 6,
        "max_epochs": 150,
        "patience": 25,
        "min_delta": 0.0005
    }

    # Device setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.backends.cudnn.benchmark = True
    print(f"Using device: {device}")

    # Data loading
    train_dataset = RNAGraphDataset(config["train_sequences"], config["train_labels"])
    val_dataset = RNAGraphDataset(config["validation_sequences"], config["validation_labels"])

    train_loader = DataLoader(
        train_dataset,
        batch_size=config["batch_size"],
        shuffle=True,
        collate_fn=lambda x: Batch.from_data_list(x),
        num_workers=4 if str(device) == 'cuda' else 0,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=config["batch_size"],
        collate_fn=lambda x: Batch.from_data_list(x),
        num_workers=4 if str(device) == 'cuda' else 0,
        pin_memory=True
    )

    # Model setup
    model = RNA3DModel(
        hidden_channels=config["hidden_dim"],
        num_layers=config["num_layers"]
    ).to(device)
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Optimizer and scheduler
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=5e-4,
        weight_decay=1e-5
    )
    scheduler = OneCycleLR(
        optimizer,
        max_lr=2e-4,
        steps_per_epoch=len(train_loader),
        epochs=config["max_epochs"],
        pct_start=0.4,
        div_factor=5,
        final_div_factor=1000
    )

    # Training loop
    best_val = float('inf')
    patience = config["patience"]

    for epoch in range(1, config["max_epochs"] + 1):
        train_loss = train_epoch(model, train_loader, optimizer, device)
        val_loss = validate(model, val_loader, device)
        scheduler.step()

        print(f"Epoch {epoch:03d} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")

        if val_loss < (best_val - config["min_delta"]):
            best_val = val_loss
            patience = config["patience"]
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_loss,
            }, "best_model.pt")
            print(f"New best model saved (Val Loss: {val_loss:.6f})")
        else:
            patience -= 1
            if patience <= 0:
                print(f"\nEarly stopping triggered at epoch {epoch}")
                print(f"Best validation loss achieved: {best_val:.6f}")
                break

    print("\nTraining completed!")

if __name__ == "__main__":
    main()

Using device: cpu
Model parameters: 3,688,963
Epoch 001 | Train Loss: 157.154622 | Val Loss: 13.349664
New best model saved (Val Loss: 13.349664)
Epoch 002 | Train Loss: 37.812583 | Val Loss: 3.356622
New best model saved (Val Loss: 3.356622)
Epoch 003 | Train Loss: 20.511656 | Val Loss: 1.159717
New best model saved (Val Loss: 1.159717)
Epoch 004 | Train Loss: 16.391667 | Val Loss: 0.760597
New best model saved (Val Loss: 0.760597)
Epoch 005 | Train Loss: 15.212419 | Val Loss: 0.737906
New best model saved (Val Loss: 0.737906)
Epoch 006 | Train Loss: 14.774716 | Val Loss: 0.864384
Epoch 007 | Train Loss: 14.471766 | Val Loss: 0.721812
New best model saved (Val Loss: 0.721812)
Epoch 008 | Train Loss: 14.251153 | Val Loss: 0.654009
New best model saved (Val Loss: 0.654009)
Epoch 009 | Train Loss: 14.152201 | Val Loss: 0.659165
Epoch 010 | Train Loss: 14.069575 | Val Loss: 0.735835
Epoch 011 | Train Loss: 13.982658 | Val Loss: 0.934914
Epoch 012 | Train Loss: 13.817139 | Val Loss: 0.6656

In [26]:
#!/usr/bin/env python3
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, Batch
import numpy as np
from torch.optim.lr_scheduler import OneCycleLR
import pandas as pd

# Enhanced Dataset with Augmentations
class RNAGraphDataset(torch.utils.data.Dataset):
    def __init__(self, sequences_file, labels_file):
        self.sequences = pd.read_csv(sequences_file)
        self.labels = pd.read_csv(labels_file)
        self.mean = None
        self.std = None
        self.graphs = self._process_data()
        self.lengths = [len(seq) for seq in self.sequences['sequence']]
        self.max_length = max(self.lengths) if self.lengths else 1

    def _process_data(self):
        graphs = []
        all_coords = []

        # First pass: collect statistics
        for _, row in self.sequences.iterrows():
            target_id = row['target_id']
            matched_labels = self.labels[self.labels['ID'].str.startswith(target_id)]
            if not matched_labels.empty and len(matched_labels) == len(row['sequence']):
                coords = matched_labels[['x_1', 'y_1', 'z_1']].values.astype(np.float32)
                all_coords.append(coords)

        all_coords = np.concatenate(all_coords, axis=0) if all_coords else np.zeros((1,3))
        self.mean = all_coords.mean(axis=0)
        self.std = all_coords.std(axis=0) + 1e-8

        # Second pass: build graphs
        for _, row in self.sequences.iterrows():
            target_id = row['target_id']
            sequence = row['sequence']
            matched_labels = self.labels[self.labels['ID'].str.startswith(target_id)]

            if matched_labels.empty or len(matched_labels) != len(sequence):
                continue

            node_feats = self._encode_sequence(sequence)
            coords = matched_labels[['x_1', 'y_1', 'z_1']].values.astype(np.float32)
            coords = (coords - self.mean) / self.std

            edge_index, edge_attr = self._build_edges(len(sequence), coords)

            graphs.append(Data(
                x=torch.tensor(node_feats, dtype=torch.float32),
                edge_index=edge_index,
                edge_attr=edge_attr,
                y=torch.tensor(coords, dtype=torch.float32),
                pos=torch.tensor(coords, dtype=torch.float32)
            ))
        return graphs

    def _encode_sequence(self, sequence):
        mapping = {
            'A': [1,0,0,0, 0.12, 0.89],
            'U': [0,1,0,0, 0.23, 0.76],
            'G': [0,0,1,0, 0.34, 0.65],
            'C': [0,0,0,1, 0.45, 0.54]
        }
        return [mapping.get(nt, [0]*6) for nt in sequence]

    def _build_edges(self, length, coords):
        edge_index = []
        edge_attr = []
        for i in range(length - 1):
            vec = coords[i+1] - coords[i]
            dist = np.linalg.norm(vec)
            edge_index.append([i, i+1])
            edge_index.append([i+1, i])
            edge_attr.extend([
                [dist, vec[0], vec[1], vec[2], 1.0],
                [dist, -vec[0], -vec[1], -vec[2], 0.0]
            ])
        return (torch.tensor(edge_index).t().contiguous(),
                torch.tensor(np.array(edge_attr), dtype=torch.float32))

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        data = self.graphs[idx]

        # Enhanced augmentations
        if torch.rand(1) < 0.5:  # 50% augmentation probability
            noise_level = 0.02 * (self.lengths[idx]/self.max_length)
            data.y += torch.randn_like(data.y) * noise_level
            data.pos += torch.randn_like(data.pos) * noise_level
            if hasattr(data, 'edge_attr'):
                data.edge_attr[:, :4] += torch.randn_like(data.edge_attr[:, :4]) * (noise_level/2)

        return data

# Enhanced Model Architecture
class RNA3DModel(nn.Module):
    def __init__(self, in_channels=6, hidden_channels=512, num_layers=6):
        super().__init__()

        # Input projection with layer norm
        self.input_proj = nn.Sequential(
            nn.Linear(in_channels, hidden_channels),
            nn.LayerNorm(hidden_channels),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.1)
        )

        # GCN layers with residual connections
        self.convs = nn.ModuleList([
            GCNConv(hidden_channels, hidden_channels, improved=True)
            for _ in range(num_layers)
        ])
        self.bns = nn.ModuleList([
            nn.BatchNorm1d(hidden_channels)
            for _ in range(num_layers)
        ])

        # Output network with skip connection
        self.output_net = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels*2),
            nn.LeakyReLU(0.1),
            nn.Dropout(0.2),
            nn.Linear(hidden_channels*2, 3)
        )

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
                nn.init.constant_(m.bias, 0)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Input projection
        x = self.input_proj(x)
        x_skip = x  # Save for skip connection

        # Message passing
        for conv, bn in zip(self.convs, self.bns):
            x = conv(x, edge_index) + x  # Residual
            x = bn(x)
            x = F.leaky_relu(x, 0.1)

        # Output with skip connection
        return self.output_net(x + x_skip)

def validate(model, loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            pred = model(data)
            total_loss += F.mse_loss(pred, data.y).item()
    return total_loss / len(loader)

def main():
    # Configuration
    config = {
        "train_sequences": "/kaggle/cleaned_train_sequences2 (1).csv",
        "train_labels": "/kaggle/train_labels1.csv",
        "validation_sequences": "/kaggle/validation_sequences.csv",
        "validation_labels": "/kaggle/validation_labels.csv",
        "batch_size": 32 if torch.cuda.is_available() else 8,
        "hidden_dim": 512,
        "num_layers": 6,
        "max_epochs": 200,
        "patience": 30,
        "min_delta": 0.0001,
        "grad_clip": 0.3
    }

    # Device setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Data loading
    train_dataset = RNAGraphDataset(config["train_sequences"], config["train_labels"])
    val_dataset = RNAGraphDataset(config["validation_sequences"], config["validation_labels"])

    train_loader = DataLoader(
        train_dataset,
        batch_size=config["batch_size"],
        shuffle=True,
        collate_fn=lambda x: Batch.from_data_list(x),
        num_workers=4 if str(device) == 'cuda' else 0,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=config["batch_size"],
        collate_fn=lambda x: Batch.from_data_list(x),
        num_workers=4 if str(device) == 'cuda' else 0,
        pin_memory=True
    )

    # Model setup
    model = RNA3DModel(
        hidden_channels=config["hidden_dim"],
        num_layers=config["num_layers"]
    ).to(device)
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Optimizer
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=5e-4,
        weight_decay=1e-5
    )

    # Scheduler
    scheduler = OneCycleLR(
        optimizer,
        max_lr=3e-4,
        steps_per_epoch=len(train_loader),
        epochs=config["max_epochs"],
        pct_start=0.3,
        div_factor=10,
        final_div_factor=1000
    )

    # Training
    best_val = float('inf')
    best_epoch = 0

    for epoch in range(1, config["max_epochs"] + 1):
        model.train()
        train_loss = 0

        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()

            pred = model(data)
            loss = F.mse_loss(pred, data.y)

            # Enhanced regularization
            loss += 0.01 * sum(p.norm(2) for p in model.parameters())
            loss += 0.001 * sum(p.abs().sum() for p in model.parameters())

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), config["grad_clip"])
            optimizer.step()
            train_loss += loss.item()

        # Validation
        val_loss = validate(model, val_loader, device)
        scheduler.step()

        # Print epoch summary
        print(f"Epoch {epoch:03d} | "
              f"Train Loss: {train_loss/len(train_loader):.6f} | "
              f"Val Loss: {val_loss:.6f} | "
              f"LR: {optimizer.param_groups[0]['lr']:.2e}")

        # Early stopping check
        if val_loss < best_val - config["min_delta"]:
            best_val = val_loss
            best_epoch = epoch
            torch.save(model.state_dict(), "best_model.pt")
            print(f"New best model saved (Val Loss: {val_loss:.6f})")

        if epoch - best_epoch > config["patience"]:
            print(f"\nEarly stopping triggered at epoch {epoch}")
            print(f"Best validation loss achieved: {best_val:.6f}")
            break

    print("\nTraining completed!")

if __name__ == "__main__":
    main()

Using device: cpu
Model parameters: 2,115,075
Epoch 001 | Train Loss: 288.483676 | Val Loss: 9.988841 | LR: 3.00e-05
New best model saved (Val Loss: 9.988841)
Epoch 002 | Train Loss: 174.345858 | Val Loss: 5.012448 | LR: 3.00e-05
New best model saved (Val Loss: 5.012448)
Epoch 003 | Train Loss: 126.142201 | Val Loss: 2.365739 | LR: 3.00e-05
New best model saved (Val Loss: 2.365739)
Epoch 004 | Train Loss: 105.116877 | Val Loss: 1.538714 | LR: 3.00e-05
New best model saved (Val Loss: 1.538714)
Epoch 005 | Train Loss: 96.516326 | Val Loss: 1.024414 | LR: 3.00e-05
New best model saved (Val Loss: 1.024414)
Epoch 006 | Train Loss: 92.858419 | Val Loss: 0.860663 | LR: 3.00e-05
New best model saved (Val Loss: 0.860663)
Epoch 007 | Train Loss: 90.685149 | Val Loss: 0.867502 | LR: 3.00e-05
Epoch 008 | Train Loss: 89.176761 | Val Loss: 0.730641 | LR: 3.00e-05
New best model saved (Val Loss: 0.730641)
Epoch 009 | Train Loss: 87.723397 | Val Loss: 0.837185 | LR: 3.00e-05
Epoch 010 | Train Loss: 86

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
import shutil

# Define destination folder path in Google Drive
drive_path = "/content/drive/MyDrive/RNA_3D_Models/"
!mkdir -p "$drive_path"

# Move the model
shutil.copy("best_model.pt", drive_path + "best_model.pt")
print(f"Model saved to {drive_path}best_model.pt")


Model saved to /content/drive/MyDrive/RNA_3D_Models/best_model.pt


In [31]:
model.load_state_dict(torch.load("/content/drive/MyDrive/RNA_3D_Models/best_model.pt"))
model.eval()


RNA3DModel(
  (input_proj): Sequential(
    (0): Linear(in_features=6, out_features=512, bias=True)
    (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Dropout(p=0.1, inplace=False)
  )
  (convs): ModuleList(
    (0-5): 6 x GCNConv(512, 512)
  )
  (bns): ModuleList(
    (0-5): 6 x BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (output_net): Sequential(
    (0): Linear(in_features=512, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.1)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=1024, out_features=3, bias=True)
  )
)

In [32]:
# Save the model's state_dict
torch.save(model.state_dict(), "best_rna3d_model.pt")


In [33]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Create a folder and save the model
drive_path = "/content/drive/MyDrive/RNA_3D_Models/"
!mkdir -p "$drive_path"

# Copy the saved model to Drive
shutil.copy("best_rna3d_model.pt", drive_path + "best_rna3d_model.pt")
print(f"Model saved to {drive_path}best_rna3d_model.pt")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model saved to /content/drive/MyDrive/RNA_3D_Models/best_rna3d_model.pt
